In [9]:
import torch
from torch import nn
from transformers import AutoModel, AutoTokenizer

In [2]:
weights = torch.load("/home/sbaruah_usc_edu/mica_text_coref/data/"
                     "word_level_coref/roberta_(e20_2021.05.02_01.16)_release.pt")

In [3]:
weights.keys()

dict_keys(['bert', 'we', 'rough_scorer', 'pw', 'a_scorer', 'sp', 'epochs_trained'])

In [5]:
for name, module in weights.items():
    if isinstance(module, dict):
        print(name)
        if name != "bert":
            for tensor_name, tensor in module.items():
                print(f"\t{tensor_name:50s}: {tensor.dtype} {tensor.shape}")
    else:
        print(f"{name}: {module}")

bert
we
	attn.weight                                       : torch.float32 torch.Size([1, 1024])
	attn.bias                                         : torch.float32 torch.Size([1])
rough_scorer
	bilinear.weight                                   : torch.float32 torch.Size([1024, 1024])
	bilinear.bias                                     : torch.float32 torch.Size([1024])
pw
	genre_emb.weight                                  : torch.float32 torch.Size([7, 20])
	distance_emb.weight                               : torch.float32 torch.Size([9, 20])
	speaker_emb.weight                                : torch.float32 torch.Size([2, 20])
a_scorer
	hidden.0.weight                                   : torch.float32 torch.Size([1024, 3132])
	hidden.0.bias                                     : torch.float32 torch.Size([1024])
	out.weight                                        : torch.float32 torch.Size([1, 1024])
	out.bias                                          : torch.float32 torch.Size([1])
sp
	ff

In [6]:
encoder = AutoModel.from_pretrained("roberta-large", add_pooling_layer=False)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
encoder.load_state_dict(weights["bert"], strict=False)

_IncompatibleKeys(missing_keys=[], unexpected_keys=['pooler.dense.weight', 'pooler.dense.bias'])

In [8]:
weights["rough_scorer"]["bilinear.bias"]

tensor([ 0.0298, -0.0126, -0.0024,  ..., -0.0076, -0.0152, -0.0162],
       device='cuda:0')

In [10]:
ll = nn.Linear(1024, 1024)

In [11]:
ll.load_state_dict(weights["rough_scorer"])

RuntimeError: Error(s) in loading state_dict for Linear:
	Missing key(s) in state_dict: "weight", "bias". 
	Unexpected key(s) in state_dict: "bilinear.weight", "bilinear.bias". 

In [12]:
type(weights["rough_scorer"])

collections.OrderedDict

In [13]:
odict = weights["rough_scorer"]

In [14]:
odict.keys()

odict_keys(['bilinear.weight', 'bilinear.bias'])

In [15]:
odict["weight"] = odict.pop("bilinear.weight")
odict["bias"] = odict.pop("bilinear.bias")

In [16]:
odict

OrderedDict([('weight',
              tensor([[ 1.7543e-01, -2.7519e-02,  1.4653e-02,  ..., -1.0604e-02,
                        2.5374e-02, -1.7066e-03],
                      [ 4.9204e-02,  1.3426e-01, -9.1869e-03,  ..., -4.4060e-03,
                        8.7263e-03, -9.5577e-03],
                      [-1.7296e-02, -1.0086e-02,  5.9279e-02,  ...,  1.6338e-05,
                       -3.0986e-02, -2.6442e-02],
                      ...,
                      [-8.4727e-04, -7.4070e-02, -3.5817e-02,  ...,  1.2996e-01,
                       -6.5809e-02, -1.6725e-02],
                      [-2.8037e-02,  2.8999e-03,  3.8483e-02,  ..., -9.9639e-03,
                        1.5081e-01, -3.2012e-02],
                      [ 7.8245e-03,  2.4031e-02, -1.3260e-03,  ...,  1.7210e-02,
                        2.2953e-02,  7.6747e-02]], device='cuda:0')),
             ('bias',
              tensor([ 0.0298, -0.0126, -0.0024,  ..., -0.0076, -0.0152, -0.0162],
                     device='cuda:0'))

In [17]:
ll.load_state_dict(odict)

<All keys matched successfully>

In [18]:
m = torch.randn((6, 10))
m

tensor([[-2.5322e+00,  3.5645e-01,  1.2496e+00, -6.6363e-01, -5.7696e-02,
         -2.4032e+00, -1.1683e+00, -1.5347e+00, -9.0168e-01, -3.4147e-01],
        [ 1.0448e+00,  8.6082e-01,  4.0752e-01,  1.2085e-01,  6.8700e-01,
         -1.9015e+00,  4.4953e-01, -1.2387e+00,  3.4712e-01, -2.3638e-01],
        [-7.8853e-01,  8.4060e-01,  1.6117e+00, -6.3626e-01, -2.2419e-01,
          1.3773e+00,  7.7333e-01,  4.2581e-01, -3.2233e-01,  7.2474e-01],
        [ 6.1090e-01, -1.0920e+00,  1.7118e+00, -1.7395e-01,  1.1789e+00,
         -6.5169e-01,  6.7840e-01,  4.3342e-01, -8.1362e-01,  5.2562e-01],
        [ 1.1133e-01, -6.5109e-02, -1.6205e-01, -5.5029e-01, -9.2431e-01,
         -8.0238e-01,  8.7520e-01, -2.2727e-01,  6.7101e-01,  1.7409e-03],
        [ 4.8888e-02,  1.2630e+00,  4.0822e-02, -1.3240e+00, -3.5563e-01,
         -2.3136e-01, -2.4115e+00,  2.2647e-01, -3.2495e-01,  2.7960e-01]])

In [19]:
(m > 0)

tensor([[False,  True,  True, False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True, False,  True, False,  True, False],
        [False,  True,  True, False, False,  True,  True,  True, False,  True],
        [ True, False,  True, False,  True, False,  True,  True, False,  True],
        [ True, False, False, False, False, False,  True, False,  True,  True],
        [ True,  True,  True, False, False, False, False,  True, False,  True]])

In [22]:
(m > 0).sum()

tensor(30)

In [21]:
(m > 0).nonzero().shape

torch.Size([30, 2])

In [23]:
(m > 0).nonzero()

tensor([[0, 1],
        [0, 2],
        [1, 0],
        [1, 1],
        [1, 2],
        [1, 3],
        [1, 4],
        [1, 6],
        [1, 8],
        [2, 1],
        [2, 2],
        [2, 5],
        [2, 6],
        [2, 7],
        [2, 9],
        [3, 0],
        [3, 2],
        [3, 4],
        [3, 6],
        [3, 7],
        [3, 9],
        [4, 0],
        [4, 6],
        [4, 8],
        [4, 9],
        [5, 0],
        [5, 1],
        [5, 2],
        [5, 7],
        [5, 9]])

In [24]:
(m > 0).nonzero(as_tuple=True)

(tensor([0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4,
         4, 5, 5, 5, 5, 5]),
 tensor([1, 2, 0, 1, 2, 3, 4, 6, 8, 1, 2, 5, 6, 7, 9, 0, 2, 4, 6, 7, 9, 0, 6, 8,
         9, 0, 1, 2, 7, 9]))

In [27]:
class MovieCoreference(nn.Module):
    """Movie screenplay coreference model
    """

    def __init__(self,
                 encoder_name: str,
                 parsetag_size: int,
                 postag_size: int,
                 nertag_size: int,
                 tag_embedding_size: int,
                 gru_nlayers: int,
                 gru_hidden_size: int,
                 gru_bidirectional: bool,
                 dropout: float) -> None:
        super().__init__()
        self.dropout = dropout

        # Word Encoder
        self.encoder = AutoModel.from_pretrained(encoder_name, 
                                                 add_pooling_layer=False)
        word_embedding_size = self.encoder.config.hidden_size
        self.attn = nn.Linear(word_embedding_size, 1)

        # Character Recognizer
        gru_input_size = word_embedding_size + 3*tag_embedding_size + 2
        gru_output_size = gru_hidden_size * (1 + gru_bidirectional)
        self.parsetag_embedding = nn.Embedding(parsetag_size, 
                                               tag_embedding_size)
        self.postag_embedding = nn.Embedding(postag_size, tag_embedding_size)
        self.nertag_embedding = nn.Embedding(nertag_size, tag_embedding_size)
        self.gru = nn.GRU(gru_input_size, gru_hidden_size,
                          num_layers=gru_nlayers, batch_first=True,
                          dropout=dropout, bidirectional=gru_bidirectional)
        self.character_recognizer = nn.Linear(gru_output_size, 2)

        # Coarse Coreference Scorer
        self.coarse_scorer = nn.Linear(word_embedding_size, word_embedding_size)

        # Pairwise Encoder
        self.genre_embedding = nn.Embedding(7, 20)
        self.distance_embedding = nn.Embedding(9, 20)
        self.speaker_embedding = nn.Embedding(2, 20)

        # Fine/Anaphoricity Coreference Scorer
        pairwise_encoding_size = 3*word_embedding_size + 3*20
        self.anaphoricity_scorer = nn.Sequential(
            nn.Linear(pairwise_encoding_size, word_embedding_size), 
            nn.LeakyReLU(), 
            nn.Dropout(self.dropout),
            nn.Linear(word_embedding_size, 1))

        # Span Predictor
        self.ffn = nn.Sequential(
            nn.Linear(word_embedding_size * 2 + 64, word_embedding_size),
            nn.ReLU(),
            nn.Dropout(self.dropout),
            nn.Linear(word_embedding_size, 256),
            nn.ReLU(),
            nn.Dropout(self.dropout),
            nn.Linear(256, 64),
        )
        self.conv = nn.Sequential(
            nn.Conv1d(64, 4, 3, 1, 1),
            nn.Conv1d(4, 2, 3, 1, 1)
        )
        self.sp_distance_embedding = nn.Embedding(128, 64)

In [28]:
model = MovieCoreference("roberta-large", 6, 5, 5, 16, 1, 256, True, 0.3)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/sbaruah_usc_edu/anaconda3/envs/coreference/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1

In [29]:
model.encoder.load_state_dict(weights["bert"], strict=False)

_IncompatibleKeys(missing_keys=[], unexpected_keys=['pooler.dense.weight', 'pooler.dense.bias'])

In [34]:
model.attn.state_dict()

OrderedDict([('weight',
              tensor([[ 0.0274, -0.0074,  0.0170,  ..., -0.0028, -0.0003,  0.0308]])),
             ('bias', tensor([0.0079]))])

In [35]:
attn_weights = weights["we"]
attn_weights

OrderedDict([('attn.weight',
              tensor([[-0.0294,  0.0403, -0.0034,  ..., -0.0082, -0.0159, -0.0712]],
                     device='cuda:0')),
             ('attn.bias', tensor([-0.0024], device='cuda:0'))])

In [36]:
attn_weights["weight"] = attn_weights.pop("attn.weight")
attn_weights["bias"] = attn_weights.pop("attn.bias")
attn_weights

OrderedDict([('weight',
              tensor([[-0.0294,  0.0403, -0.0034,  ..., -0.0082, -0.0159, -0.0712]],
                     device='cuda:0')),
             ('bias', tensor([-0.0024], device='cuda:0'))])

In [37]:
model.attn.load_state_dict(attn_weights)

<All keys matched successfully>

In [38]:
model.coarse_scorer.state_dict()

OrderedDict([('weight',
              tensor([[-0.0012, -0.0090,  0.0198,  ...,  0.0059, -0.0276, -0.0110],
                      [-0.0181,  0.0176, -0.0229,  ...,  0.0203, -0.0258, -0.0184],
                      [-0.0262,  0.0248, -0.0231,  ...,  0.0295, -0.0271,  0.0060],
                      ...,
                      [-0.0188, -0.0187,  0.0175,  ..., -0.0174,  0.0008, -0.0183],
                      [-0.0161,  0.0238, -0.0280,  ..., -0.0187, -0.0077,  0.0038],
                      [ 0.0194, -0.0022,  0.0209,  ...,  0.0256,  0.0307, -0.0286]])),
             ('bias',
              tensor([ 0.0171,  0.0281, -0.0117,  ...,  0.0109, -0.0084,  0.0265]))])

In [39]:
model.genre_embedding.state_dict()

OrderedDict([('weight',
              tensor([[ 0.3629, -0.0317, -1.5324,  0.1600,  0.4448, -1.6962,  0.1380, -0.4193,
                        0.8495,  0.3328,  0.3605, -0.6877, -0.9906, -1.3488, -0.1052, -1.1878,
                        0.0500,  0.0109,  0.1253, -1.0734],
                      [-0.8249,  0.6914,  0.4895,  1.4984, -0.7223,  0.5873, -0.4211, -0.8235,
                       -0.0543, -2.6446, -0.3753,  0.8424,  0.1767, -0.1740, -0.9725, -1.1067,
                        0.6391,  0.2110,  0.8932,  0.3329],
                      [-0.5997,  0.1488, -1.2311,  0.7732,  1.2663, -0.0118,  1.1087,  1.2458,
                        0.1896, -1.7539,  0.6302, -1.0682,  0.2268,  0.6963,  0.2854, -0.2986,
                        0.7904,  0.0242, -1.0095, -1.2411],
                      [-1.4591, -1.0668, -0.7136, -1.0395,  0.5830, -0.7281,  1.2340,  1.0527,
                        0.4809,  0.3712, -1.2334,  1.8825, -0.0613,  0.4623,  0.9979, -1.0711,
                        0.0243, -0.8

In [40]:
model.anaphoricity_scorer.state_dict()

OrderedDict([('0.weight',
              tensor([[-0.0051,  0.0125,  0.0166,  ...,  0.0075,  0.0118, -0.0066],
                      [ 0.0081,  0.0166, -0.0081,  ...,  0.0043, -0.0017,  0.0057],
                      [-0.0067, -0.0130,  0.0028,  ...,  0.0003, -0.0038,  0.0155],
                      ...,
                      [-0.0001, -0.0148, -0.0022,  ...,  0.0175, -0.0129, -0.0136],
                      [ 0.0081, -0.0014,  0.0015,  ..., -0.0029, -0.0101, -0.0071],
                      [-0.0125, -0.0109,  0.0078,  ...,  0.0142,  0.0116, -0.0080]])),
             ('0.bias',
              tensor([ 0.0112,  0.0117, -0.0118,  ..., -0.0012,  0.0024,  0.0057])),
             ('3.weight',
              tensor([[ 0.0188,  0.0092, -0.0285,  ..., -0.0025, -0.0029, -0.0098]])),
             ('3.bias', tensor([-0.0025]))])

In [42]:
model.ffn.state_dict().keys()

odict_keys(['0.weight', '0.bias', '3.weight', '3.bias', '6.weight', '6.bias'])

In [43]:
model.anaphoricity_scorer.state_dict().keys()

odict_keys(['0.weight', '0.bias', '3.weight', '3.bias'])